# Setup

In [ ]:
import numpy as np
import open3d as o3d
from sklearn import cluster
import pandas as pd

from PointCloudDataFrame import PointCloudDataFrame

In [ ]:
filename = 'original-cropped.pts'
pcd_raw = o3d.io.read_point_cloud(f'input/{filename}')
df_raw = PointCloudDataFrame.from_pcd(pcd_raw)

In [ ]:
pcd = pcd_raw.voxel_down_sample(voxel_size=0.001)  # 1mm
df = PointCloudDataFrame.from_pcd(pcd)

In [ ]:
df_filter1 = df[(df['s'] < 0.1) & (df['v'] > 0.2)]
pcd_filter1 = df_filter1.to_pcd()

In [ ]:
_, ind = pcd_filter1.remove_radius_outlier(nb_points=56, radius=0.005)
pcd_inliers = pcd_filter1.select_by_index(ind)
pcd_inliers.paint_uniform_color([0.8, 0.8, 0.8])
df_inliers = PointCloudDataFrame.from_pcd(pcd_inliers)

In [ ]:
model = cluster.DBSCAN(eps=0.003, min_samples=10)
model.fit(pcd_inliers.points)

In [ ]:
labels = model.labels_ 
num_labels = len(set(labels).difference({-1}))
print(num_labels)

centroids = []
for i in range(num_labels):
    centroids.append(
        np.mean(df_inliers[labels==i][['x', 'y', 'z']], axis=0)
    )

pcd_centroids = o3d.geometry.PointCloud()
pcd_centroids.points = o3d.utility.Vector3dVector(centroids)

# o3d.visualization.draw_geometries([pcd_inliers, pcd_centroids])

In [ ]:
for i in range(num_labels):
    point = centroids[i]
    filter = ((df_raw['x'] - point['x'])**2 + (df_raw['y'] - point['y'])**2 + (df_raw['z'] - point['z'])**2) < 0.006**2

    pcd_temp = df_raw[filter].to_pcd()
    pcd_temp.paint_uniform_color([0, 0.8, 0])

    o3d.visualization.draw_geometries([pcd_temp, pcd])

In [ ]:
label_names = ['E105', 'E63', 'E80', 'E34', 'E44', 'E110', 'E11', 'E23', 'E82', 'E108',
               'E51', 'COM', 'E40', 'E58', 'E62', 'E87', 'E36', 'E16', 'E79', 'E7',
               'E119', 'E50', 'E115', 'E4', 'E93', 'E18', 'E21', 'E125', 'E38', 'E33',
               'E15', 'E20', 'E123', 'E8', 'E117', 'E10', 'E27', 'E14', 'E49', 'E19',
               'E76', 'E70', 'E39', 'E9', 'E111', 'E101', 'E102', 'E3', 'E109', 'E103',
               'E106', 'E118', 'E59', 'E43', 'E61', 'E45', 'E81', 'E114', 'E60', 'E72',
               'E35', 'E75', 'E28', 'E13', 'VERF', 'E101', 'E29', 'E89', 'E84', '',
               'E99', 'E37', 'E121', 'E77', 'E92', 'E64', 'E65', 'E91', 'E30', 'E46',
               'E67', 'E83', 'E12', 'E122', 'E95', 'E124', 'E104', 'E71', 'E24', 'E98',
               'E54', 'E2', 'E42', 'E97', 'E128', 'E66', 'E86', 'E41', 'E57', 'E53',
               'E55', 'E1', 'E113', 'E81', 'E107', 'E88', 'E73', 'E43', 'E85', 'E78',
               'E5', 'E6', 'E69', 'E17', 'E116', 'E90', 'E96', 'E31', '', 'E52',
               'E120', 'E47', 'E112', '', '', '']

In [ ]:
total_labels = {'VERF', 'COM'}.union(f'E{i}' for i in range(1, 129))
missing_labels = total_labels.difference(label_names)
print(missing_labels)

In [ ]:
df_raw['label'] = ''
for i, l in enumerate(label_names):
    if l:
        point = centroids[i]
        filter = ((df_raw['x'] - point['x'])**2 + (df_raw['y'] - point['y'])**2 + (df_raw['z'] - point['z'])**2) < 0.006**2
        df_raw.loc[filter, 'label'] = l
df_raw

point = np.mean(
    pd.DataFrame([
        centroids[label_names.index('E99')],
        centroids[label_names.index('E101')],
        centroids[label_names.index('E101')],
        centroids[label_names.index('E101')],
        centroids[label_names.index('E107')],
    ]), axis=0
)
filter = ((df_raw['x'] - point['x'])**2 + (df_raw['y'] - point['y'])**2 + (df_raw['z'] - point['z'])**2) < 0.006**2

pcd_temp = df_raw[filter].to_pcd()
pcd_temp.paint_uniform_color([0, 0.8, 0])

o3d.visualization.draw_geometries([pcd_temp, pcd])

In [ ]:
df_raw.write_label(f'label/{filename.split(".")[0]}.txt')